<a href="https://colab.research.google.com/github/rafaelreisramos/oracle-one/blob/main/challenge-telecom-x-parte_2/TelecomX_BR_parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [181]:
import pandas as pd

In [182]:
df = pd.read_csv('https://raw.githubusercontent.com/rafaelreisramos/oracle-one/refs/heads/main/challenge-telecom-x-parte_2/dados/telecom_x.csv')
df.head()

,Churn,Gender,Senior_Citizen,Partner,Dependents,Tenure,Phone_Service,Multiple_Lines,Internet_Service,Online_Security,Online_Backup,Device_Protection,Tech_Support,Streaming_TV,Streaming_Movies,Contract,Paperless_Billing,Payment_Method,Charges_Monthly,Charges_Total
0,False,Female,False,True,True,9,True,No,DSL,No,Yes,No,Yes,Yes,No,One year,True,Mailed check,65.6,593.30
1,False,Male,False,False,False,9,True,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,False,Mailed check,59.9,542.40
2,True,Male,False,False,False,4,True,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,True,Electronic check,73.9,280.85
3,True,Male,True,True,False,13,True,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,True,Electronic check,98.0,1237.85
4,True,Female,True,True,False,3,True,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,True,Mailed check,83.9,267.40


In [183]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7032 entries, 0 to 7031
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Churn              7032 non-null   bool   
 1   Gender             7032 non-null   object 
 2   Senior_Citizen     7032 non-null   bool   
 3   Partner            7032 non-null   bool   
 4   Dependents         7032 non-null   bool   
 5   Tenure             7032 non-null   int64  
 6   Phone_Service      7032 non-null   bool   
 7   Multiple_Lines     7032 non-null   object 
 8   Internet_Service   7032 non-null   object 
 9   Online_Security    7032 non-null   object 
 10  Online_Backup      7032 non-null   object 
 11  Device_Protection  7032 non-null   object 
 12  Tech_Support       7032 non-null   object 
 13  Streaming_TV       7032 non-null   object 
 14  Streaming_Movies   7032 non-null   object 
 15  Contract           7032 non-null   object 
 16  Paperless_Billing  7032 

Para os algoritmos de Machine Learning as variáveis de entrada devem ser mapeadas em valores numéricos. Vamos então mapear as colunas booleanas e a coluna `Gender` da seguinte forma:
- `{True: 1, False: 0}`
- `{'Male': 1, 'Female': 0}`

In [184]:
df = df.replace({True: 1, False: 0, 'Male': 1, 'Female': 0}).infer_objects(copy=False)

/tmp/ipython-input-873424863.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({True: 1, False: 0, 'Male': 1, 'Female': 0}).infer_objects(copy=False)


Vamos dividir nossos dados em `x` e `y`

In [185]:
x = df.drop('Churn', axis = 1)
y = df['Churn']

Agora vamos codificar as colunas do tipo `object`

In [186]:
object_cols = list(df.select_dtypes(include='object').columns)
cols = x.columns

In [187]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

one_hot = make_column_transformer((
        OneHotEncoder(),
        object_cols
    ),
    remainder = 'passthrough',
    sparse_threshold=0)

x = one_hot.fit_transform(x)
df_encoded = pd.DataFrame(x, columns = one_hot.get_feature_names_out(cols)).join(df['Churn'])

Vamos verificar a proporção e balanceamento dos dados.

In [189]:
y.value_counts(normalize = True)

,proportion
Churn,
0,0.734215
1,0.265785


Vamos balancear os dados com `Oversampling`.

In [190]:
from imblearn.over_sampling import SMOTE

oversample = SMOTE(random_state = 47)
x_balanced, y_balanced = oversample.fit_resample(x, y)